In [ ]:
!pip install pandas_market_calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, LeakyReLU, MaxPooling2D, concatenate, LSTM, Reshape, Dense
from keras.callbacks import EarlyStopping

import pandas_market_calendars as mcal

2025-05-25 23:58:48.575515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748217528.791891      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748217528.857016      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def moving_average(x, k):
    return np.convolve(x, np.ones(k), 'valid') / k

def compute_mid_price_changes(daily_data_dict, date_list, k):
    all_changes = []
    for date in date_list:
        price_bid = daily_data_dict[date][:, 0]
        size_bid = daily_data_dict[date][:, 1]
        price_ask = daily_data_dict[date][:, 2]
        size_ask = daily_data_dict[date][:, 3]
        mid_price = (price_ask * size_bid + price_bid * size_ask) / (size_ask + size_bid)

        future_avg = moving_average(mid_price, k)[1:]  # Skip current step
        current = mid_price[:-k]
        pct_change = (future_avg - current) / current
        all_changes.extend(pct_change)
    return np.array(all_changes)

In [ ]:
df = pd.read_csv("/kaggle/input/eth-large/combined_orderbooks_eth_large.csv")

In [ ]:
df

,timestamp,bid_price_1,bid_size_1,ask_price_1,ask_size_1,bid_price_2,bid_size_2,ask_price_2,ask_size_2,bid_price_3,...,ask_price_8,ask_size_8,bid_price_9,bid_size_9,ask_price_9,ask_size_9,bid_price_10,bid_size_10,ask_price_10,ask_size_10
0,1745798400,1790.31,38.59,1790.32,15.04,1790.30,0.68,1790.34,0.08,1790.28,...,1790.43,0.02,1790.21,0.02,1790.45,0.04,1790.20,1.12,1790.47,0.02
1,1745798401,1790.31,38.71,1790.32,14.93,1790.30,0.68,1790.34,0.08,1790.28,...,1790.43,0.02,1790.21,0.02,1790.45,0.04,1790.20,1.12,1790.47,0.02
2,1745798402,1790.31,18.98,1790.32,27.39,1790.28,0.02,1790.33,0.03,1790.26,...,1790.41,1.00,1790.20,1.12,1790.43,0.02,1790.19,0.16,1790.45,0.02
3,1745798403,1790.31,27.70,1790.32,9.89,1790.28,0.02,1790.34,0.08,1790.26,...,1790.43,0.02,1790.20,1.12,1790.45,0.02,1790.19,0.16,1790.47,0.02
4,1745798404,1790.31,27.58,1790.32,10.34,1790.28,0.02,1790.34,0.08,1790.26,...,1790.43,0.02,1790.20,1.12,1790.45,0.02,1790.19,0.16,1790.47,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863996,1746662396,1810.48,52.96,1810.49,62.14,1810.47,1.53,1810.50,0.01,1810.45,...,1810.58,5.06,1810.36,0.02,1810.59,16.34,1810.35,6.29,1810.60,1.97
863997,1746662397,1810.48,6.68,1810.49,74.63,1810.47,0.02,1810.50,0.01,1810.45,...,1810.59,16.34,1810.36,0.02,1810.60,1.97,1810.35,6.29,1810.61,0.03
863998,1746662398,1810.48,19.02,1810.49,55.80,1810.46,0.02,1810.50,0.01,1810.45,...,1810.59,16.34,1810.36,8.30,1810.60,1.97,1810.35,6.29,1810.61,0.03
863999,1746662399,1810.26,4.74,1810.27,93.51,1810.25,0.02,1810.30,2.95,1810.24,...,1810.38,0.01,1810.13,3.87,1810.40,1.97,1810.12,0.17,1810.41,0.30


In [ ]:
df = df.sort_values(by=['timestamp']).reset_index(drop=True)

In [ ]:
df['date'] = pd.to_datetime(df['timestamp'], unit='s')
df['date'] = df['date'].dt.date
df['date'] = df['date'].apply(lambda x: str(x))

In [ ]:
df = df[df['date'] != '2025-05-08'].reset_index(drop=True)

In [ ]:
numeric_columns = [col for col in df.columns if col not in ['timestamp', 'date']]

In [ ]:
dates_str_list = list(df['date'].unique())

In [ ]:
daily_data_dict = {}

In [ ]:
for i in range(len(dates_str_list)):
    date = dates_str_list[i]
    if date not in daily_data_dict.keys():
        daily_data_dict[date] = np.array(df[df['date']==dates_str_list[i]][numeric_columns])

In [ ]:
mid_price_changes = compute_mid_price_changes(daily_data_dict, ['2025-04-28'], 8)

In [ ]:
alpha = np.quantile(mid_price_changes, 0.3)

In [ ]:
alpha

-6.495761631192181e-05

In [ ]:
daily_data_dict.pop('2025-04-28')

array([[1.79031e+03, 3.85900e+01, 1.79032e+03, ..., 1.12000e+00,
        1.79047e+03, 2.00000e-02],
       [1.79031e+03, 3.87100e+01, 1.79032e+03, ..., 1.12000e+00,
        1.79047e+03, 2.00000e-02],
       [1.79031e+03, 1.89800e+01, 1.79032e+03, ..., 1.60000e-01,
        1.79045e+03, 2.00000e-02],
       ...,
       [1.79890e+03, 1.36890e+02, 1.79891e+03, ..., 2.00000e-02,
        1.79901e+03, 2.00000e-02],
       [1.79890e+03, 1.39910e+02, 1.79891e+03, ..., 2.00000e-02,
        1.79901e+03, 2.00000e-02],
       [1.79890e+03, 1.38780e+02, 1.79891e+03, ..., 2.00000e-02,
        1.79901e+03, 2.00000e-02]])

In [ ]:
dates_str_list = dates_str_list[1:]

In [ ]:
normalization_mean_dict = {}
normalization_stddev_dict = {}

In [ ]:
for i in range(5,len(dates_str_list)):
    date = dates_str_list[i]

    if (date not in normalization_mean_dict.keys()) or (date not in normalization_stddev_dict.keys()):
        look_back_dates_list = dates_str_list[(i-5):i]
        prev_5_day_orderbook_np = None
        for look_back_date in look_back_dates_list:
            if prev_5_day_orderbook_np is None:
                prev_5_day_orderbook_np = daily_data_dict[look_back_date]
            else:
                prev_5_day_orderbook_np = np.vstack((prev_5_day_orderbook_np, daily_data_dict[look_back_date]))


        price_mean = prev_5_day_orderbook_np[:,range(0,prev_5_day_orderbook_np.shape[1],2)].mean()
        price_std = prev_5_day_orderbook_np[:,range(0,prev_5_day_orderbook_np.shape[1],2)].std()
        size_mean = prev_5_day_orderbook_np[:,range(1,prev_5_day_orderbook_np.shape[1],2)].mean()
        size_std = prev_5_day_orderbook_np[:,range(1,prev_5_day_orderbook_np.shape[1],2)].std()

        normalization_mean_dict[date] = np.repeat([[price_mean,size_mean]], 20, axis=0).flatten()
        normalization_stddev_dict[date] = np.repeat([[price_std,size_std]], 20, axis=0).flatten()

In [ ]:
daily_norm_data_dict = {}

In [ ]:
for i in range(5,len(dates_str_list)):
    date = dates_str_list[i]
    if date not in daily_norm_data_dict.keys():
        daily_norm_data_dict[date] = (daily_data_dict[date] - normalization_mean_dict[date])/ normalization_stddev_dict[date]

In [ ]:
list(daily_data_dict.keys())[5:]

['2025-05-04', '2025-05-05', '2025-05-06', '2025-05-07']

In [ ]:
def generate_labels(k, alpha, daily_data_dict):
    daily_label_dict = {}
    for date in list(daily_data_dict.keys())[5:]:
        price_bid = daily_data_dict[date][:,0]
        size_bid = daily_data_dict[date][:,1]
        price_ask = daily_data_dict[date][:,2]
        size_ask = daily_data_dict[date][:,3]
        mid_price = (price_ask * size_bid + price_bid * size_ask) / (size_ask + size_bid)
        future_k_avg_mid_price = moving_average(mid_price, k)[1:]
        print(future_k_avg_mid_price.shape, mid_price[:-k].shape)
        change_pct = (future_k_avg_mid_price - mid_price[:-k])/mid_price[:-k]
        print(change_pct, len(change_pct))
        y_label = (-(change_pct < -alpha).astype(int)) + (change_pct > alpha).astype(int)

        daily_label_dict[date] = y_label.reshape(-1,1)
    return daily_label_dict

In [ ]:
def generate_X_y(k, alpha, timestamp_per_sample, daily_norm_data_dict, daily_data_dict):
    #k is the number of future timesteps used to generate the label y
    data_x = None
    for date in daily_norm_data_dict.keys():
        if data_x is None:
            data_x = daily_norm_data_dict[date].copy()[:-k,:]
        else:
            data_x = np.vstack((data_x, daily_norm_data_dict[date][:-k,:]))

    daily_label_dict = generate_labels(k, alpha, daily_data_dict)
    data_y = None
    for date in daily_label_dict.keys():
        if data_y is None:
            data_y = daily_label_dict[date].copy()
        else:
            data_y = np.vstack((data_y, daily_label_dict[date]))

    [N, P_x] = data_x.shape

    x = np.zeros([(N-timestamp_per_sample+1), timestamp_per_sample, P_x])

    for i in range(N-timestamp_per_sample+1):
        x[i] = data_x[i:(i+timestamp_per_sample), :]

    x = x.reshape(x.shape + (1,))
    y = data_y[(timestamp_per_sample-1):]
    y = to_categorical(y, 3)

    return x, y

In [ ]:
daily_norm_data_dict.pop('2025-05-07')
daily_data_dict.pop('2025-05-07')

array([[1.81636e+03, 5.87400e+01, 1.81637e+03, ..., 4.00000e-02,
        1.81650e+03, 1.97400e+01],
       [1.81679e+03, 1.34600e+01, 1.81680e+03, ..., 4.60000e-01,
        1.81692e+03, 3.36000e+00],
       [1.81681e+03, 2.49000e+00, 1.81682e+03, ..., 2.00000e-02,
        1.81693e+03, 3.40000e-01],
       ...,
       [1.81048e+03, 6.68000e+00, 1.81049e+03, ..., 6.29000e+00,
        1.81061e+03, 3.00000e-02],
       [1.81048e+03, 1.90200e+01, 1.81049e+03, ..., 6.29000e+00,
        1.81061e+03, 3.00000e-02],
       [1.81026e+03, 4.74000e+00, 1.81027e+03, ..., 1.70000e-01,
        1.81041e+03, 3.00000e-01]])

In [ ]:
X,y = generate_X_y(k=8, alpha=alpha, timestamp_per_sample=100,
                   daily_norm_data_dict= daily_norm_data_dict,
                   daily_data_dict = daily_data_dict)

(86392,) (86392,)
[ 5.57455492e-05  1.01256770e-05  1.72004573e-05 ... -4.25251081e-05
 -4.80992974e-05 -5.47959905e-05] 86392
(86392,) (86392,)
[-1.89716698e-04 -2.51378867e-04 -3.24225802e-04 ...  3.04242716e-05
 -5.76103946e-08  1.98654638e-06] 86392
(86392,) (86392,)
[ 1.94291057e-04 -1.46386669e-06 -2.07872586e-05 ...  6.15573640e-05
  8.43943060e-05  1.04291292e-04] 86392


In [ ]:
y[:, 0].sum(), y[:, 1].sum(), y[:, 2].sum()

(137151.0, 61099.0, 60827.0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, shuffle=False)

In [ ]:
lookback_timestep = 100
feature_num = 40

#Conv param
conv_filter_num = 16

#Inception module param
inception_num = 32

#LSTM param
LSTM_num = 64

#Activation param
leaky_relu_alpha = 0.01

# categorical crossentropy loss
loss = 'categorical_crossentropy'

# ADAM is used
learning_rate = 0.01
adam_epsilon = 1
optimizer = Adam(learning_rate=learning_rate, epsilon=1)

# accuracy is used for stopping training
metrics = ['accuracy']

#max epoch num is not specified in paper, use 120 because paper mentions training stops at about 100 epochs
num_epoch = 10000
#stop training when validation accuracy does not improve for 20 epochs
stop_epoch_num = 20
#mini-batch size 32 from paper
batch_size = 32

I0000 00:00:1748217611.494308      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748217611.495050      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [ ]:
def initiate_DeepLOB_model(lookback_timestep, feature_num, conv_filter_num, inception_num, LSTM_num, leaky_relu_alpha,
                          loss, optimizer, metrics):

    input_tensor = Input(shape=(lookback_timestep, feature_num, 1))

    # Conv block1
    print(input_tensor.shape)
    conv_layer1 = Conv2D(conv_filter_num, (1,2), strides=(1, 2))(input_tensor)
    print(conv_layer1.shape)
    conv_layer1 =LeakyReLU(alpha=leaky_relu_alpha)(conv_layer1)
    print(conv_layer1.shape)
    conv_layer1 = Conv2D(conv_filter_num, (4,1), padding='same')(conv_layer1)
    conv_first1 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer1)
    print(conv_layer1.shape)
    conv_layer1 = Conv2D(conv_filter_num, (4,1), padding='same')(conv_layer1)
    conv_layer1 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer1)
    print(conv_layer1.shape)

    # Conv block2
    conv_layer2 = Conv2D(conv_filter_num, (1,2), strides=(1, 2))(conv_layer1)
    conv_layer2 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer2)
    print(conv_layer2.shape)
    conv_layer2 = Conv2D(conv_filter_num, (4,1), padding='same')(conv_layer2)
    conv_layer2 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer2)
    print(conv_layer2.shape)
    conv_layer2 = Conv2D(conv_filter_num, (4,1), padding='same')(conv_layer2)
    conv_layer2 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer2)
    print(conv_layer2.shape)

    # Conv block3
    conv_layer3 = Conv2D(conv_filter_num, (1,10))(conv_layer2)
    conv_layer3 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer3)
    print(conv_layer3.shape)
    conv_layer3 = Conv2D(conv_filter_num, (4,1), padding='same')(conv_layer3)
    conv_layer3 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer3)
    print(conv_layer3.shape)
    conv_layer3 = Conv2D(conv_filter_num, (4,1), padding='same')(conv_layer3)
    conv_layer3 = LeakyReLU(alpha=leaky_relu_alpha)(conv_layer3)
    print(conv_layer3.shape)

    # Inception module
    inception_module1 = Conv2D(inception_num, (1,1), padding='same')(conv_layer3)
    inception_module1 = LeakyReLU(alpha=leaky_relu_alpha)(inception_module1)
    print(inception_module1.shape)
    inception_module1 = Conv2D(inception_num, (3,1), padding='same')(inception_module1)
    inception_module1 = LeakyReLU(alpha=leaky_relu_alpha)(inception_module1)
    print(inception_module1.shape)

    inception_module2 = Conv2D(inception_num, (1,1), padding='same')(conv_layer3)
    inception_module2 = LeakyReLU(alpha=leaky_relu_alpha)(inception_module2)
    print(inception_module2.shape)
    inception_module2 = Conv2D(inception_num, (5,1), padding='same')(inception_module2)
    inception_module2 = LeakyReLU(alpha=leaky_relu_alpha)(inception_module2)
    print(inception_module2.shape)

    inception_module3 = MaxPooling2D((3,1), strides=(1,1), padding='same')(conv_layer3)
    print(inception_module3.shape)
    inception_module3 = Conv2D(inception_num, (1,1), padding='same')(inception_module3)
    print(inception_module3.shape)
    inception_module3 = LeakyReLU(alpha=leaky_relu_alpha)(inception_module3)
    print(inception_module3.shape)

    inception_module_final = concatenate([inception_module1, inception_module2, inception_module3], axis=3)
    print(inception_module_final.shape)
    inception_module_final = Reshape((inception_module_final.shape[1], inception_module_final.shape[3]))(inception_module_final)
    print(inception_module_final.shape)

    # LSTM
    LSTM_output = LSTM(LSTM_num)(inception_module_final)
    print(LSTM_output.shape)

    # Fully Connected Layer with softmax activation function for output
    model_output = Dense(3, activation='softmax')(LSTM_output)
    print(model_output.shape)

    DeepLOB_model = Model(inputs=input_tensor, outputs= model_output)
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1)

    DeepLOB_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return DeepLOB_model

In [ ]:
DeepLOB_model = initiate_DeepLOB_model(lookback_timestep, feature_num, conv_filter_num, inception_num, LSTM_num, leaky_relu_alpha,
                          loss, optimizer, metrics)

(None, 100, 40, 1)
(None, 100, 20, 16)
(None, 100, 20, 16)
(None, 100, 20, 16)
(None, 100, 20, 16)
(None, 100, 10, 16)
(None, 100, 10, 16)
(None, 100, 10, 16)
(None, 100, 1, 16)
(None, 100, 1, 16)
(None, 100, 1, 16)
(None, 100, 1, 32)
(None, 100, 1, 32)
(None, 100, 1, 32)
(None, 100, 1, 32)
(None, 100, 1, 16)
(None, 100, 1, 32)
(None, 100, 1, 32)
(None, 100, 1, 96)
(None, 100, 96)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


(None, 64)
(None, 3)


In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience = stop_epoch_num, verbose=1)
DeepLOB_model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=2, validation_data=(X_test, y_test), callbacks = [es])

Epoch 1/100


I0000 00:00:1748217634.120621     107 cuda_dnn.cc:529] Loaded cuDNN version 90300


6477/6477 - 99s - 15ms/step - accuracy: 0.5293 - loss: 1.0100 - val_accuracy: 0.5284 - val_loss: 1.0058
Epoch 2/100
6477/6477 - 89s - 14ms/step - accuracy: 0.5296 - loss: 1.0047 - val_accuracy: 0.5284 - val_loss: 1.0058
Epoch 3/100
6477/6477 - 90s - 14ms/step - accuracy: 0.5301 - loss: 0.9988 - val_accuracy: 0.5285 - val_loss: 0.9905
Epoch 4/100
6477/6477 - 89s - 14ms/step - accuracy: 0.5334 - loss: 0.9771 - val_accuracy: 0.5259 - val_loss: 0.9738
Epoch 5/100
6477/6477 - 89s - 14ms/step - accuracy: 0.5379 - loss: 0.9634 - val_accuracy: 0.5402 - val_loss: 0.9543
Epoch 6/100
6477/6477 - 88s - 14ms/step - accuracy: 0.5414 - loss: 0.9537 - val_accuracy: 0.5429 - val_loss: 0.9501
Epoch 7/100
6477/6477 - 90s - 14ms/step - accuracy: 0.5450 - loss: 0.9489 - val_accuracy: 0.5457 - val_loss: 0.9473
Epoch 8/100
6477/6477 - 89s - 14ms/step - accuracy: 0.5477 - loss: 0.9460 - val_accuracy: 0.5465 - val_loss: 0.9434
Epoch 9/100
6477/6477 - 88s - 14ms/step - accuracy: 0.5479 - loss: 0.9441 - val_accu

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Предсказания
y_pred_probs = DeepLOB_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Отчёт по классам
print(classification_report(y_true, y_pred, target_names=['Down', 'Neutral', 'Up'], digits=4))

1620/1620 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step
              precision    recall  f1-score   support

        Down     0.6451    0.7860    0.7086     27377
     Neutral     0.5525    0.4290    0.4830     12186
          Up     0.5560    0.4082    0.4708     12253

    accuracy                         0.6127     51816
   macro avg     0.5845    0.5411    0.5541     51816
weighted avg     0.6022    0.6127    0.5993     51816

